In [1]:
import pandas as pd

In [2]:
a=pd.read_csv("iso3_to_wb_name.csv").rename(columns={"country":"wb_name"})

In [13]:
b=pd.read_csv("country_codes_extra.csv")
b.columns

Index(['Short name', 'Official name', 'ISO3', 'ISO2', 'UNI', 'UNDP', 'FAOSTAT',
       'GAUL'],
      dtype='object')

In [10]:
r=pd.merge(a,b,left_on = 'county_ID', right_on = 'countyid')
r

KeyError: 'ISO3'

In [8]:
r.to_csv("iso3_WBname_iso2_uni_undp_fasotat_gaul.csv",index=False)